In [33]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import pandas as pd
import chromadb
import uuid


In [9]:
job_text = """
Munich, Germany
Munich, Germany
Intern / Working Student
Working Student/ Intern
Intern / Working Student
Engineering
(Intern / Working Student) NLP Engineer, LLM Agents
As a NLP Engineer, you will work with the tech team to further develop our AI Agent and expand its capabilities. This is a very high impact role where you will shape the product and work at the very edge of whats possible with the state-of-the-art technologies.

About Us:

We are KNOWRON GmbH, a deep tech startup based in Munich, Germany. Widespread loss of knowledge and an ageing workforce require a fundamental change in manufacturing. That's why we are building a Large Language Model (LLM) based platform to assist desk-less workers anywhere, 24/7.

As a NLP Engineer, you will work with the tech team to further develop our AI Agents capabilities. This is a very high impact role where you will shape the product and work at the very edge of whats possible with the state-of-the-art technologies.

‍

Key responsibilities

Implement and test new skills for our AI Agent.
Evaluate the performance and accuracy of the pipeline to ensure high value for the user
Write clean and maintainable code with sensible testing that you would enjoy taking over from a colleague.
Proactively communicate, document, and share your approach, progress, results, and challenges.
Your profile

You have built a project with LLM technology.
You are currently pursuing your Masters degree in Computer Science, Electrical Engineering or any other technical related field.
You are proficient in Python and are comfortable working with REST APIs
You have experience solving NLP problems and are comfortable with the usuals in this space (Spacy, Gensim, Cohere/ OpenAI / Gemini etc).
You are results-oriented and have strong organizational & communication skills
You have a go-getter attitude. You are proactive about finding solutions to problems.
You are fluent in English
What's in it for you?

Steep Learning Curve: Join a funded early stage startup and learn the product discovery and development process from the inside out.
Mentorship: We do regular 1:1s, retrospectives and support you to learn new things to grow personally and professionally.
Hybrid Work: We are a company built during the covid era, hybrid work is a core part of our culture.
Personal Development Budget: You get a 1000€ yearly budget for personal growth and development.
Team Events & Fitness: We provide healthy snacks, drinks, and exciting team activities and an Urban Sports Club membership
Hit us up!

‍

Please send your CV, your LinkedIn profile and your earliest possible starting date to jobs@knowron.com. We are looking forward to your application and will try to get back to you as soon as possible!

‍

KNOWRON is an equal opportunity employer. We embrace and celebrate diversity and are committed to creating an inclusive environment for all employees. We are open to all groups of people without regard to age, color, national origin, race, religion, gender, sex, sexual orientation, gender identity and/or expression, marital status, or any other legally protected characteristics.
"""

In [40]:
def extract_job_text(text):
    llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        groq_api_key="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",
        temperature="0.8"
    )

    extract_prompt = PromptTemplate(
        input_variables=["text"],
        template="""
        Extract detail information such as `role`, `experience`, `skills`, `description` from {text} and made into JSON format.
        """
    )

    chain_extract = extract_prompt | llm
    response = chain_extract.invoke({"text": text})
    
    return response.content

In [41]:
res = extract_job_text(job_text)

In [42]:
def parse_extract_info(extract_output):
    json_parser = JsonOutputParser()
    json_res = json_parser.parse(extract_output)
    return json_res

In [43]:
json_res = parse_extract_info(res)

In [44]:
json_res

{'role': 'NLP Engineer, LLM Agents',
 'experience': 'Intern / Working Student',
 'location': 'Munich, Germany',
 'description': "As a NLP Engineer, you will work with the tech team to further develop our AI Agent and expand its capabilities. This is a very high impact role where you will shape the product and work at the very edge of what's possible with the state-of-the-art technologies.",
 'skills': ['LLM technology',
  'Python',
  'REST APIs',
  'NLP problem-solving',
  'Spacy',
  'Gensim',
  'Cohere/OpenAI/Gemini',
  'English fluency'],
 'responsibilities': ['Implement and test new skills for our AI Agent.',
  'Evaluate the performance and accuracy of the pipeline to ensure high value for the user',
  'Write clean and maintainable code with sensible testing that you would enjoy taking over from a colleague.',
  'Proactively communicate, document, and share your approach, progress, results, and challenges.'],
 'requirements': ["Currently pursuing a Master's degree in Computer Scienc

In [30]:
portforlio_dir = "/Users/yu/Desktop/Code/ColdEmailGenerator/coldemail_practice/my_portfolio.csv"

portfolio_df = pd.read_csv(portforlio_dir, encoding="latin-1", delimiter=";")
portfolio_df

,Project Name,Project Description,Project Links
0,"ZipfÕs Law, Text Generation Using A N-gram Mod...",NaN,https://github.com/yyuuccii/CoLi_1/blob/main/C...
1,Comparative Analysis of BERT Variants on Senti...,"Evaluate BERT-base, RoBERTa, DistilBERT, and A...",https://drive.google.com/file/d/13RJ4inZYD4eEd...
2,Comparative Analysis of Sentiment Analysis Tec...,Compare sentiment analysis results from NLTK/V...,https://drive.google.com/file/d/1hioeSSo9hQ_g_...
3,IT Service Ticket Classification with DistilBERT,Classify IT service tickets into categories by...,https://github.com/yyuuccii/TextClassification...
4,"Yoga Pose Classification with custom CNN, ResN...","Classify yoga poses with a custom CNN, ResNet5...",https://github.com/yyuuccii/ImageClassificatio...


In [56]:
client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in portfolio_df.iterrows():
        document = f"{row['Project Name']} - {row['Project Description']}"
        collection.add(documents=[document],
                       metadatas={"links": row["Project Links"]},
                       ids=[str(uuid.uuid4())]
                       )

In [57]:
description = json_res["description"]
responsibilities = json_res["responsibilities"]
requirements = json_res["requirements"]

In [61]:
query_text = []
query_text = str(description) + str(responsibilities) + str(requirements)

'As a NLP Engineer, you will work with the tech team to further develop our AI Agent and expand its capabilities. This is a very high impact role where you will shape the product and work at the very edge of what\'s possible with the state-of-the-art technologies.[\'Implement and test new skills for our AI Agent.\', \'Evaluate the performance and accuracy of the pipeline to ensure high value for the user\', \'Write clean and maintainable code with sensible testing that you would enjoy taking over from a colleague.\', \'Proactively communicate, document, and share your approach, progress, results, and challenges.\']["Currently pursuing a Master\'s degree in Computer Science, Electrical Engineering or any other technical related field.", \'Have built a project with LLM technology.\', \'Proficient in Python and comfortable working with REST APIs\', \'Experience solving NLP problems and comfortable with the usuals in this space\', \'Results-oriented with strong organizational & communicati

In [62]:
links = collection.query(query_texts=query_text,
                         n_results=5
                         )

In [63]:
links

{'ids': [['efe507dd-951f-419d-9c6e-a972ff2c635c',
   '447e40d4-f59f-4ef6-a532-e90d78050690',
   '67d6964f-d905-4c87-a4d5-a902b5d66d5c',
   '1c46e88f-6600-46f3-964b-9e2146fc8dcd',
   '5454d921-c054-426c-8134-aad1c3d3c4c7']],
 'distances': [[1.5215454653640508,
   1.6042790629939843,
   1.807712897476488,
   1.835226161623634,
   1.9274019281605625]],
 'metadatas': [[{'links': 'https://drive.google.com/file/d/1hioeSSo9hQ_g_aQTISfR2UZtYFH4GR57/view?usp=sharing'},
   {'links': 'https://drive.google.com/file/d/13RJ4inZYD4eEdOVK9lHhEQ64VAs1WgSs/view'},
   {'links': 'https://github.com/yyuuccii/TextClassification_ITService/tree/main'},
   {'links': 'https://github.com/yyuuccii/CoLi_1/blob/main/CoLi_1.ipynb'},
   {'links': 'https://github.com/yyuuccii/ImageClassification_YogaPose'}]],
 'embeddings': None,
 'documents': [['Comparative Analysis of Sentiment Analysis Techniques and Preprocessing Impact - Compare sentiment analysis results from NLTK/VADER, SpaCy/Textblob, and HuggingFace, examinin

In [93]:
def write_email(links, query_text):
    llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        groq_api_key="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",
        temperature=0.8
    )

    prompt_email = PromptTemplate(
        input_variables=["links", "query_text"],
        template="""
        You are Yuci Chen, a master's student studying Computational Linguistics in Saarland University.
        You are applying for a internship/workstudent position in a AI company called Knowron.
        Write a application letter, where you look for the requirements in {query_text} 
        and add the most relevant ones from the following links to showcase portfolio: {links}, but don't write down details about the projects, just provide the links.
        Write it CREATIVELY and not sound like AI generated. 
        Write in the end "P.S. This letter was generated by a Large Language Model (LLM) model."
        Natural tone.
        Do not provide a preamble.
        """
    )

    chain_email = prompt_email | llm
    response = chain_email.invoke({"query_text": query_text, "links": links})
    # output = response.content
    # output = output.replace("\n", "")

    return response

In [94]:
email = write_email(query_text, links)
print(email.content)

Dear Hiring Manager at Knowron,

I'm beyond excited to apply for the internship/work student position at Knowron, where I can contribute my skills and passion for NLP to drive innovation and growth. As a master's student in Computational Linguistics at Saarland University, I'm confident that my technical expertise and experience make me an ideal fit for this role.

With a strong foundation in Python and experience working with REST APIs, I'm well-equipped to tackle complex NLP challenges. My academic pursuits have honed my problem-solving skills, and I'm eager to apply my knowledge to real-world problems. I'm particularly drawn to Knowron's commitment to pushing the boundaries of AI technology and shaping the product with cutting-edge innovations.

As someone who thrives in a results-driven environment, I'm excited about the prospect of working on high-impact projects that prioritize user value. I'm a firm believer in the importance of clear communication, documentation, and collaborat